## 특징 검출 방법

#### 특징을 검출하기 위해서는 특징 검출기와 디스크립터를 이용해야 한다.

- 특징 검출기: FastFeatureDetector, MSER, SimpleBlobDetector, GFTTDetector
- 특징 검출기 & 디스크립터: BRISK, ORB, KAZE, AKAZE, SIFT

#### 특징 검출 및 특징 표시는 다음과 같은 공통적인 과정을 통해 이루어진다.

- cv2.검출기이름.create(): 검출기 객체 생성
- 만든객체.detect(영상): 만든 검출기로 이미지에서 특징을 detect 한다.
- cv2.drawKeypoints(영상, 발견한 특징, 색): 발견한 특징을 영상에 나타낸다.


## FastFeatureDectector

In [ ]:
import cv2
import numpy as np

src = cv2.imread('./data/chessBoard.jpg')
gray = cv2.cvtColor(src,cv2.COLOR_BGR2GRAY)

fastF = cv2.FastFeatureDetector.create(threshold = 30)
kp = fastF.detect(gray)
dst = cv2.drawKeypoints(gray,kp,None,color=(0,0,255))
print('len(kp)=',len(kp))

fastF.setNonmaxSuppression(False)
kp2 = fastF.detect(gray)
dst2 = cv2.drawKeypoints(src,kp2,None,color=(0,0,255))
print('len(kp2)=',len(kp2))

dst3 = src.copy()
points = cv2.KeyPoint_convert(kp)
points = np.int32(points)
print('len(points)=', len(points))

for cx,cy in points:
    cv2.circle(dst3,(cx,cy),3,color=(255,0,0),thickness=-1)
        
cv2.imshow('dst', dst)
cv2.imshow('dst2', dst2)
cv2.imshow('dst3', dst3)
cv2.waitKey()
cv2.destroyAllWindows()  

In [13]:
src = cv2.imread('./data/chessBoard.jpg')
gray = cv2.cvtColor(src,cv2.COLOR_BGR2GRAY)

fastF = cv2.FastFeatureDetector.create() # 더 확실한 점을 검출하고 싶으면 threshold를 높게 잡으면 됨
kp = fastF.detect(gray)
dst = cv2.drawKeypoints(gray,kp,None,color=(255,0,0))
print('len(kp)=',len(kp))

kp = sorted(kp, key = lambda f: f.response, reverse = True)
cv2.drawKeypoints(gray,kp[:10], dst, color=(0,0,255), flags = cv2.DRAW_MATCHES_FLAGS_DRAW_OVER_OUTIMG)
cv2.imshow('dst', dst)

kp2 = list(filter(lambda f: f.response > 50, kp))
print('len(kp2)=',len(kp2))

dst2 = cv2.drawKeypoints(gray, kp2, None, color=(0,0,255))
cv2.imshow('dst2',dst2)

def distance(f1,f2):
    x1,y1 = f1.pt
    x2,y2 = f2.pt
    return np.sqrt((x2-x1)**2+(y2-y1)**2)

def filteringByDistance(kp,distE = 0.5):
    size = len(kp)
    mask = np.arange(1,size+1).astype(np.bool8)
    for i,f1 in enumerate(kp):
        if not mask[i]:
            continue
        else:
            for j,f2 in enumerate(kp):
                if i==j:
                    continue
                if distance(f1,f2) < distE:
                    mask[j] = False
    np_kp = np.array(kp)
    return list(np_kp[mask])

kp3 = filteringByDistance(kp2, 30)
print('len(kp3)=', len(kp3))
dst3 = cv2.drawKeypoints(gray, kp3, None, color=(0,0,255))
cv2.imshow('dst3', dst3)
cv2.waitKey()
cv2.destroyAllWindows()  

len(kp)= 419
len(kp2)= 315
len(kp3)= 81


## MSER 특징 검출

In [8]:
src = cv2.imread('./data/chessBoard.jpg')
src = cv2.medianBlur(src,ksize=9)
gray = cv2.cvtColor(src,cv2.COLOR_BGR2GRAY)

mserF = cv2.MSER_create(30)  # 10으로 설정한 delta 값은 커질수록 더 적은 개수의 영역이 검출된다.
kp = mserF.detect(gray)
print('len(kp)=',len(kp))
dst = cv2.drawKeypoints(gray,kp,None,color=(0,0,255))

dst2 = dst.copy()
regions,bboxes = mserF.detectRegions(gray)
hulls = [cv2.convexHull(p.reshape(-1,1,2)) for p in regions]
cv2.polylines(dst2,hulls,True,(0,255,0))
cv2.imshow('dst2', dst2)

dst3 = dst.copy()
for i,pts in enumerate(regions):
    box = cv2.fitEllipse(pts)
    x,y,w,h = bboxes[i]
    cv2.rectangle(dst3,(x,y),(x+w,y+h),(0,255,0))
    
cv2.imshow('dst3', dst3)
cv2.waitKey()
cv2.destroyAllWindows()      

len(kp)= 210


## SimpleBlobDetector

In [9]:
src = cv2.imread('./data/chessBoard.jpg')
#gray = cv2.cvtColor(src,cv2.COLOR_BGR2GRAY)

params = cv2.SimpleBlobDetector_Params()
params.blobColor = 0
params.thresholdStep=5
params.minThreshold=20
params.maxThreshold = 100
params.minDistBetweenBlobs = 5
params.filterByArea = True
params.minArea = 25
params.maxArea = 5000
params.filterByConvexity = True
params.minConvexity = 0.89

blobF = cv2.SimpleBlobDetector_create()
kp = blobF.detect(gray)
print('len(kp)=',len(kp))
dst = cv2.drawKeypoints(src,kp,None,color=(255,0,0))

for f in kp:
    r = int(f.size/2)
    cx,cy = f.pt
    cv2.circle(dst,(round(cx),round(cy)),r,(0,0,255),2)
    
cv2.imshow('dst', dst)
cv2.waitKey()
cv2.destroyAllWindows()  

len(kp)= 35


## GFTTDetector 특징 검출

In [10]:
src = cv2.imread('./data/chessBoard.jpg')
gray = cv2.cvtColor(src,cv2.COLOR_BGR2GRAY)

goodF = cv2.GFTTDetector_create()
kp = goodF.detect(gray)
print('len(kp)=',len(kp))
dst = cv2.drawKeypoints(gray,kp,None,color=(0,0,255))

goodF2 = cv2.GFTTDetector_create(maxCorners=150,
                                qualityLevel=0.1,
                                minDistance=2,
                                useHarrisDetector=True)

kp2 = goodF2.detect(gray)
print('len(kp2)=',len(kp2))
dst2 = cv2.drawKeypoints(gray,kp2,None,color=(255,0,0))

cv2.imshow('dst', dst)
cv2.imshow('dst2', dst2)
cv2.waitKey()
cv2.destroyAllWindows()  

len(kp)= 541
len(kp2)= 150


## ORB 특징 검출 및 디스크립터

In [15]:
def distance(f1, f2):
    x1, y1 = f1.pt
    x2, y2 = f2.pt
    return np.sqrt((x2-x1)**2 + (y2-y1)**2)

def filteringByDistance(kp,distE = 0.5):
    size = len(kp)
    mask = np.arange(1,size+1).astype(np.bool8)
    for i,f1 in enumerate(kp):
        if not mask[i]:
            continue
        else:
            for j,f2 in enumerate(kp):
                if i==j:
                    continue
                if distance(f1,f2) < distE:
                    mask[j] = False
    np_kp = np.array(kp)
    return list(np_kp[mask])

src = cv2.imread('./data/cornerTest.jpg')
gray = cv2.cvtColor(src,cv2.COLOR_BGR2GRAY)
gray = cv2.GaussianBlur(gray, (5,5), 0.0)

orbF = cv2.ORB_create(scoreType = 1)
kp = orbF.detect(gray)
print('len(kp)=', len(kp))
dst = cv2.drawKeypoints(gray, kp, None, color = (0,0,255))
cv2.imshow('dst', dst)

kp = sorted(kp, key = lambda f: f.response, reverse = True)
filtered_kp = list(filter(lambda f: f.response > 50, kp))
filtered_kp = filteringByDistance(kp, 10)
print('len(filtered_kp)=', len(filtered_kp))

kp, des = orbF.compute(gray, filtered_kp)
print('des.shape=', des.shape)
print('des=', des)

dst2 = cv2.drawKeypoints(gray, filtered_kp, None, color=(0,0,255))
for f in filtered_kp:
    x, y = f.pt
    size = f.size
    rect = ((x,y), (size, size), f.angle)
    box = cv2.boxPoints(rect).astype(np.int32)
    cv2.polylines(dst2, [box], True, (0,255,0), 2)
    cv2.circle(dst2, (round(x), round(y)), round(f.size/2), (255, 0, 0), 2)

cv2.imshow('dst2', dst2)
cv2.waitKey()
cv2.destroyAllWindows()  


len(kp)= 61
len(filtered_kp)= 8
des.shape= (8, 32)
des= [[ 72  48  56  96  32  77  81  16 105 168  52   8 159  23  64  50 132 212
   93   8 136  96 240   0 195 249  33  48  66 128  66  35]
 [104  48  56  96  32  77  81  16 105 168  52   8 159  23  64  50 132 212
  221   8 136  96 240   0 195 249  33  48  64 128  66  35]
 [104  48  56  96  33  77  81  16 105 168  52   8 159  23  64  50 132 212
   93   8 136  96 240   0 195 249  33  48  66 128  66  35]
 [104  48  56  96  32  77  81  16 105 136  52   8 159  23   0  48 132  20
   93   8 137  96 176   0 195 249  32  16  64 128  70  51]
 [ 72  48  56  96  32  77  81  16 105 136  52   8 159  23   0  48 132  20
   93   8 136  96 176   0 195 249  32  16  66 128  70  51]
 [104  48  56  96  33  77  81  16 105 168  52   8 159  23   0  48 132 149
  221   8 136  96 240   0 195 249  32  48  66 128  70  51]
 [  2 141   2 158  20 131 170 230  22   1  88   2  32 234  48  64 121 106
   19 129   2 186   8 255  32  38 138 136  21  14  48 136]
 [  2 141   2

In [18]:
def distance(f1, f2):
    x1, y1 = f1.pt
    x2, y2 = f2.pt
    return np.sqrt((x2-x1)**2 + (y2-y1)**2)

def filteringByDistance(kp,distE = 0.5):
    size = len(kp)
    mask = np.arange(1,size+1).astype(np.bool8)
    for i,f1 in enumerate(kp):
        if not mask[i]:
            continue
        else:
            for j,f2 in enumerate(kp):
                if i==j:
                    continue
                if distance(f1,f2) < distE:
                    mask[j] = False
    np_kp = np.array(kp)
    return list(np_kp[mask])

src = cv2.imread('./data/chessBoard.jpg')
gray = cv2.cvtColor(src,cv2.COLOR_BGR2GRAY)
gray = cv2.GaussianBlur(gray, (5,5), 0.0)

fastF = cv2.FastFeatureDetector.create(threshold = 30)
mserF = cv2.MSER_create(10)
blobF = cv2.SimpleBlobDetector_create()
goodF = cv2.GFTTDetector_create(maxCorners = 20, minDistance = 10)

kp = fastF.detect(gray)
print('len(kp)=', len(kp))

filtered_kp = filteringByDistance(kp, 10)
print('len(filtered_kp)=', len(filtered_kp))
dst = cv2.drawKeypoints(gray, filtered_kp, None, color=(0,0,255))
cv2.imshow('dst', dst)

orbF = cv2.ORB_create()
filtered_kp, des = orbF.compute(gray, filtered_kp)
print('des.shape=', des.shape)
print('des=', des)


dst2 = cv2.drawKeypoints(gray, filtered_kp, None, color=(0,0,255))
for f in filtered_kp:
    x, y = f.pt
    size = f.size
    rect = ((x,y), (size, size), f.angle)
    box = cv2.boxPoints(rect).astype(np.int32)
    cv2.polylines(dst2, [box], True, (0,255,0), 2)
    cv2.circle(dst2, (round(x), round(y)), round(f.size/2), (255, 0, 0), 2)

cv2.imshow('dst2', dst2)
cv2.waitKey()
cv2.destroyAllWindows()  


len(kp)= 359
len(filtered_kp)= 122
des.shape= (82, 32)
des= [[213  88 127 ... 179  33 121]
 [198 162 156 ...  68 133  67]
 [212  72  63 ... 179 100  56]
 ...
 [ 33  67  65 ... 178  80  56]
 [  2 166 132 ...  68 189 218]
 [134  67  14 ...  55  13  86]]


## BRISK

In [20]:
def distance(f1, f2):
    x1, y1 = f1.pt
    x2, y2 = f2.pt
    return np.sqrt((x2-x1)**2 + (y2-y1)**2)

def filteringByDistance(kp,distE = 0.5):
    size = len(kp)
    mask = np.arange(1,size+1).astype(np.bool8)
    for i,f1 in enumerate(kp):
        if not mask[i]:
            continue
        else:
            for j,f2 in enumerate(kp):
                if i==j:
                    continue
                if distance(f1,f2) < distE:
                    mask[j] = False
    np_kp = np.array(kp)
    return list(np_kp[mask])

src = cv2.imread('./data/cornerTest.jpg')
gray = cv2.cvtColor(src,cv2.COLOR_BGR2GRAY)
gray = cv2.GaussianBlur(gray, (5,5), 0.0)

briskF = cv2.BRISK_create()
kp = briskF.detect(gray)
print('len(kp)=', len(kp))
dst = cv2.drawKeypoints(gray, filtered_kp, None, color=(0,0,255))
cv2.imshow('dst', dst)

kp = sorted(kp, key = lambda f: f.response, reverse = True)
filtered_kp = list(filter(lambda f: f.response > 50, kp))
filtered_kp = filteringByDistance(kp, 10)
print('len(filtered_kp)=', len(filtered_kp))

kp, des = briskF.compute(gray, filtered_kp)
print('des.shape=', des.shape)
print('des=', des)

dst2 = cv2.drawKeypoints(gray, filtered_kp, None, color=(0,0,255))
for f in filtered_kp:
    x, y = f.pt
    size = f.size
    rect = ((x,y), (size, size), f.angle)
    box = cv2.boxPoints(rect).astype(np.int32)
    cv2.polylines(dst2, [box], True, (0,255,0), 2)
    cv2.circle(dst2, (round(x), round(y)), round(f.size/2), (255, 0, 0), 2)

cv2.imshow('dst2', dst2)
cv2.waitKey()
cv2.destroyAllWindows()  


len(kp)= 22
len(filtered_kp)= 8
des.shape= (8, 64)
des= [[252 255 231 251 125  64   0   0   0 199 159 255 244 211  78  50 137 100
  148 177 140 231  60  14   0   0   0 128  25 111  60 161   4  16  64   0
    1   4 144   0 134  24 147  49   0   0   0 154 219 101  48   0  16   0
    4  12   2   0 129   1  82 141  25   0]
 [128 255 255 231   1   0   0   0   0   0   0   0   0 199 255 255 191 255
  192   1   0   0   0   0   0   0   0   0   0   0   0 128  24 103 188 241
  142  17 131  24   0   0   0   0   0   0   0   0   0   0   0  64 187 205
  199 199 209 104   0   0   0   0   0   0]
 [128 255 255 231   1   0   0   0   0   0   0   0   0 199 255 255 191 255
  195  33  12   0   0   0   0   0   0   0   0   0   0 128  24 103 188 241
  142  17 131  24   0   0   0   0   0   0   0   0   0   0  64 115 187 205
  199 199 241 106   0   0   0   0   0   0]
 [160 255 255 227   0 128  64   0   0   0   0 224 112 199 255 255 255 255
  195   1  12   0   0   0   0   0   0   0   0   0   0 128  24 103 188 241
 

In [22]:
def distance(f1, f2):
    x1, y1 = f1.pt
    x2, y2 = f2.pt
    return np.sqrt((x2-x1)**2 + (y2-y1)**2)

def filteringByDistance(kp,distE = 0.5):
    size = len(kp)
    mask = np.arange(1,size+1).astype(np.bool8)
    for i,f1 in enumerate(kp):
        if not mask[i]:
            continue
        else:
            for j,f2 in enumerate(kp):
                if i==j:
                    continue
                if distance(f1,f2) < distE:
                    mask[j] = False
    np_kp = np.array(kp)
    return list(np_kp[mask])

src = cv2.imread('./data/chessBoard.jpg')
gray = cv2.cvtColor(src,cv2.COLOR_BGR2GRAY)
gray = cv2.GaussianBlur(gray, (5,5), 0.0)

fastF = cv2.FastFeatureDetector.create(threshold = 30)
mserF = cv2.MSER_create(10)
blobF = cv2.SimpleBlobDetector_create()
goodF = cv2.GFTTDetector_create(maxCorners = 20, minDistance = 10)

kp = fastF.detect(gray)
print('len(kp)=', len(kp))

filtered_kp = filteringByDistance(kp, 10)
print('len(filtered_kp)=', len(filtered_kp))
dst = cv2.drawKeypoints(gray, filtered_kp, None, color=(0,0,255))
cv2.imshow('dst', dst)

briskF = cv2.BRISK_create()
filtered_kp, des = briskF.compute(gray, filtered_kp)
print('des.shape=', des.shape)
print('des=', des)


dst2 = cv2.drawKeypoints(gray, filtered_kp, None, color=(0,0,255))
for f in filtered_kp:
    x, y = f.pt
    size = f.size
    rect = ((x,y), (size, size), f.angle)
    box = cv2.boxPoints(rect).astype(np.int32)
    cv2.polylines(dst2, [box], True, (0,255,0), 2)
    cv2.circle(dst2, (round(x), round(y)), round(f.size/2), (255, 0, 0), 2)

cv2.imshow('dst2', dst2)
cv2.waitKey()
cv2.destroyAllWindows()  

len(kp)= 359
len(filtered_kp)= 122
des.shape= (102, 64)
des= [[176 255 255 ... 246 239 255]
 [240 255 255 ... 189 153  50]
 [252 255 255 ...  13   9   0]
 ...
 [180 255 255 ...   0  64 192]
 [160 123 223 ... 127 111 207]
 [180 123 223 ...   8  16   0]]


# KAZE, AKAZE 특징 검출 및 디스크립터

In [24]:
def distance(f1, f2):
    x1, y1 = f1.pt
    x2, y2 = f2.pt
    return np.sqrt((x2-x1)**2 + (y2-y1)**2)

def filteringByDistance(kp,distE = 0.5):
    size = len(kp)
    mask = np.arange(1,size+1).astype(np.bool8)
    for i,f1 in enumerate(kp):
        if not mask[i]:
            continue
        else:
            for j,f2 in enumerate(kp):
                if i==j:
                    continue
                if distance(f1,f2) < distE:
                    mask[j] = False
    np_kp = np.array(kp)
    return list(np_kp[mask])

src = cv2.imread('./data/cornerTest.jpg')
gray = cv2.cvtColor(src,cv2.COLOR_BGR2GRAY)
gray = cv2.GaussianBlur(gray, (5,5), 0.0)

kazeF = cv2.KAZE_create()
akazeF = cv2.AKAZE_create()
kp = kazeF.detect(gray)
print('len(kp)=', len(kp))
dst = cv2.drawKeypoints(gray, kp, None, color=(0,0,255))
cv2.imshow('dst', dst)

kp = sorted(kp, key = lambda f: f.response, reverse = True)
filtered_kp = filteringByDistance(kp, 10)
print('len(filtered_kp)=', len(filtered_kp))

kp, des = kazeF.compute(gray, filtered_kp)
print('des.shape=', des.shape)
print('des.dtype=', des.dtype)
print('des=', des)

dst2 = cv2.drawKeypoints(gray, filtered_kp, None, color=(0,0,255))
for f in filtered_kp:
    x, y = f.pt
    size = f.size
    rect = ((x,y), (size, size), f.angle)
    box = cv2.boxPoints(rect).astype(np.int32)
    cv2.polylines(dst2, [box], True, (0,255,0), 2)
    cv2.circle(dst2, (round(x), round(y)), round(f.size/2), (255, 0, 0), 2)

cv2.imshow('dst2', dst2)
cv2.waitKey()
cv2.destroyAllWindows()  

len(kp)= 26
len(filtered_kp)= 10
des.shape= (10, 64)
des.dtype= float32
des= [[ 1.65240962e-07  1.00125305e-06  1.65240962e-07  1.00125305e-06
   6.97695532e-06  9.09818846e-05  1.18943535e-05  9.09818846e-05
  -6.54680161e-06  7.51692060e-05  1.33824933e-05  7.51692060e-05
  -5.50376384e-08  7.16649282e-08  5.50376384e-08  7.16649282e-08
   3.52473557e-02  2.13576168e-01  3.52473557e-02  2.13576168e-01
   1.14334067e-02  3.91715825e-01  9.76881385e-02  3.91715825e-01
  -1.18473507e-01  1.22508407e-01  1.32241771e-01  1.22508407e-01
  -6.82395708e-04  1.51138316e-04  6.82395708e-04  1.51138316e-04
   2.06976198e-02  1.25414178e-01  2.06976198e-02  1.25414178e-01
  -3.04565188e-02  1.31364360e-01  6.62879869e-02  1.31364360e-01
  -3.77436489e-01  8.74077603e-02  3.80755633e-01  8.74077603e-02
  -6.09615864e-03  1.01248350e-03  6.09615864e-03  1.01248350e-03
   1.12920952e-06  6.84227780e-06  1.12920952e-06  6.84227780e-06
  -5.51517541e-03  9.10298142e-04  5.51520986e-03  9.10298142e-04

# SIFT

In [26]:
def distance(f1, f2):
    x1, y1 = f1.pt
    x2, y2 = f2.pt
    return np.sqrt((x2-x1)**2 + (y2-y1)**2)

def filteringByDistance(kp,distE = 0.5):
    size = len(kp)
    mask = np.arange(1,size+1).astype(np.bool8)
    for i,f1 in enumerate(kp):
        if not mask[i]:
            continue
        else:
            for j,f2 in enumerate(kp):
                if i==j:
                    continue
                if distance(f1,f2) < distE:
                    mask[j] = False
    np_kp = np.array(kp)
    return list(np_kp[mask])

src = cv2.imread('./data/cornerTest.jpg')
gray = cv2.cvtColor(src,cv2.COLOR_BGR2GRAY)

siftF = cv2.SIFT_create(edgeThreshold = 80)
kp = siftF.detect(gray)
print('len(kp)=', len(kp))

kp = sorted(kp, key = lambda f: f.response, reverse = True)
filtered_kp = filteringByDistance(kp, 10)
print('len(filtered_kp)=', len(filtered_kp))

kp, des = siftF.compute(gray, filtered_kp)
print('des.shape=', des.shape)
print('des.dtype=', des.dtype)
print('des=', des)

dst = cv2.drawKeypoints(gray, filtered_kp, None, color=(0,0,255))
for f in filtered_kp:
    x, y = f.pt
    size = f.size
    rect = ((x,y), (size, size), f.angle)
    box = cv2.boxPoints(rect).astype(np.int32)
    cv2.polylines(dst, [box], True, (0,255,0), 2)
    cv2.circle(dst, (round(x), round(y)), round(f.size/2), (255, 0, 0), 2)

cv2.imshow('dst', dst)
cv2.waitKey()
cv2.destroyAllWindows()  

len(kp)= 21
len(filtered_kp)= 10
des.shape= (10, 128)
des.dtype= float32
des= [[11.  0.  0. ...  0.  0.  0.]
 [ 0.  0.  0. ... 14.  6.  1.]
 [ 0.  0.  0. ...  1.  0.  0.]
 ...
 [ 0.  0.  0. ...  0.  0.  0.]
 [ 0.  0.  0. ...  0.  0.  0.]
 [ 0.  0.  0. ...  0.  0.  0.]]


# 디스크립터를 이용한 특징 매칭

In [31]:
src1 = cv2.imread('./data/book1.jpg')
src2 = cv2.imread('./data/book2.jpg')
img1 = cv2.cvtColor(src1,cv2.COLOR_BGR2GRAY)
img2 = cv2.cvtColor(src2,cv2.COLOR_BGR2GRAY)

orbF = cv2.ORB_create(nfeatures = 100)
kp1, des1 = orbF.detectAndCompute(img1, None)
kp2, des2 = orbF.detectAndCompute(img2, None)

bf = cv2.BFMatcher_create(cv2.NORM_HAMMING, crossCheck = True)
matches = bf.match(des1, des2)

matches = sorted(matches,key=lambda m:m.distance)
print('len(matches)=', len(matches))
for i,m in enumerate(matches[:3]):
    print('matches[{}]=(queryIdx:{}, trainIdx:{}, distance:{})'
          .format(i, m.queryIdx, m.trainIdx, m.distance))
 
minDist = matches[0].distance
good_matches = list(filter(lambda m: m.distance < 5*minDist, matches))
print('len(good_matches)=',len(good_matches))
if len(good_matches) < 5:
    print('sorry,too small good matches')
    exit()
    
dst = cv2.drawMatches(img1,kp1,img2,kp2,good_matches,None,flags=2)
cv2.imshow('dst', dst)

src1_pts = np.float32([kp1[m.queryIdx].pt for m in good_matches])
src2_pts = np.float32([kp2[m.queryIdx].pt for m in good_matches])
    
H,mask = cv2.findHomography(src1_pts,src2_pts,cv2.RANSAC,3.0)
mask_matches = mask.ravel().tolist()
    
h,w = img1.shape
pts = np.float32([[0,0],[0,h-1],[w-1,h-1],[w-1,0]]).reshape(-1,1,2)
pts2 = cv2.perspectiveTransform(pts,H)
src2 = cv2.polylines(src2,[np.int32(pts2)],True,(255,0,0),2)
    
draw_params = dict(matchColor=(0,255,0),singlePointColor=None,matchesMask = mask_matches,flags=2)
dst2 = cv2.drawMatches(src1,kp1,src2,kp2,good_matches,None,**draw_params)

cv2.imshow('dst2', dst2)
cv2.waitKey()
cv2.destroyAllWindows()  

len(matches)= 38
matches[0]=(queryIdx:4, trainIdx:30, distance:38.0)
matches[1]=(queryIdx:1, trainIdx:10, distance:40.0)
matches[2]=(queryIdx:18, trainIdx:11, distance:42.0)
len(good_matches)= 38
